# Sum types

**`Sum types`** are a way to restrict the possible states of a value to a specific set of types, making edge cases explicit.

Then we can use the **`isinstance`** function to check if an object is an instance of the subclasses that were defined as the possible types.

In the following example, I create two classes that represents two possible ways for a document: parsed and error by trying to parse. This could look strange, but in real production, a function is committed to determinate in which object should a doc be instantiated.

We use an empty parent class (`MaybeParsed` in this case) to group these types together. This acts as a conceptual label that allows type checkers and developers to know that `Parsed` and `ParseError` belong to the same family of possible results. This class could implement properties that can be inherited, like `timestamp`, for example.

This is useful because it is cleaner to represent errors as data (e.g. the `ParseError` class) rather than **`raise exceptions`** here and there. 

In [11]:
class MaybeParsed:
    pass

class Parsed(MaybeParsed):
    def __init__(self, doc_name, text):
        self.doc_name= doc_name
        self.text = text


class ParseError(MaybeParsed):
    def __init__(self, doc_name, err):
        self.doc_name = doc_name
        self.err = err

'''
Example of using isinstance later:
if isinstance(object, Parsed):
        return True
'''

'\nExample of using isinstance later:\nif isinstance(object, Parsed):\n        return True\n'

In [10]:
run_cases = [
    Parsed("why_fp.txt", "Because we're better than everyone else"),
    ParseError("why_fp.docx", "Can't handle weird windows files"),
]

submit_cases = run_cases + [
    Parsed("why_fp.md", "Because we're better than everyone else"),
    ParseError("why_fp.pdf", "Can't handle weird adobe files"),
]


def test(obj):
    print("---------------------------------")
    print(f"Testing properties of {obj.doc_name}...")
    if isinstance(obj, Parsed):
        if not obj.text:
            print(f"Expecting .text to be non-empty")
            print("Fail")
            return False
        if not obj.doc_name:
            print(f"Expecting .doc_name to be non-empty")
            print("Fail")
            return False
    elif isinstance(obj, ParseError):
        if not obj.err:
            print(f"Expecting .err to be non-empty")
            print("Fail")
            return False
        if not obj.doc_name:
            print(f"Expecting .doc_name to be non-empty")
            print("Fail")
            return False
    else:
        raise ValueError(f"unknown class type for: {obj}")
    print("Pass")
    return True


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Testing properties of why_fp.txt...
Pass
---------------------------------
Testing properties of why_fp.docx...
Pass
---------------------------------
Testing properties of why_fp.md...
Pass
---------------------------------
Testing properties of why_fp.pdf...
Pass
============= PASS ==============
4 passed, 0 failed


# Enums

Using `isinstance` works, but **`enums`** make it better. 

**`Enum`** is a class that restricts the possible values of a set of values. In the following example, the Doctype variable creates an object on the Enum class with the name **`Doctype`** and the values as second parameters. If I try to print another format (.invalid, for example), the program would raise an Exception

In [21]:
from enum import Enum

Doctype = Enum('Doctype',['PDF','TXT','DOCX','MD','HTMl'])

#The code above is the same as this one:
class Doctype(Enum): #Inheritance from the Enum class
    PDF = 1
    TXT = 2
    DOCX = 3
    MD = 4
    HTML = 5

print(Doctype.PDF)

Doctype.PDF


In [22]:
run_cases = [
    (lambda: Doctype.PDF, "Doctype.PDF", False),
    (lambda: Doctype.TXT, "Doctype.TXT", False),
    (lambda: Doctype.DOCX, "Doctype.DOCX", False),
    (lambda: Doctype.MD, "Doctype.MD", False),
]

submit_cases = run_cases + [
    (lambda: Doctype.HTML, "Doctype.HTML", False),
    (lambda: Doctype.Invalid, "Doctype.Invalid", True),
]


def test(func, name, is_err):
    print("---------------------------------")
    print(f"Checking value: {name}")
    try:
        val = func()
        print(f"...Valid enum value!")
        return not is_err
    except Exception as e:
        print(f"...Invalid enum value!")
        return is_err


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            print("Pass")
            passed += 1
        else:
            print("Fail")
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Checking value: Doctype.PDF
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.TXT
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.DOCX
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.MD
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.HTML
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.Invalid
...Invalid enum value!
Pass
============= PASS ==============
6 passed, 0 failed


Because Python does not support **`sum types`**, we should force the raise of an Exception if a value is invalid:

In [23]:
def color_to_hex(color):
    if color == Color.GREEN:
        return '#00FF00'
    elif color == Color.BLUE:
        return '#0000FF'
    elif color == Color.RED:
        return '#FF0000'
    # handle the case where the color is invalid
    raise Exception('unknown color')

# Match

The **match** statement is cleaner than use a series of `if/else/elif`  statements when working with a fixed set of possible values.
In the following example, I use tuples to match two values

In [32]:
from enum import Enum

class DocFormat(Enum):
    PDF = 1
    TXT = 2
    MD = 3
    HTML = 4

def convert_format(content, from_format, to_format):
    #The value we want to compare is set after the match keyword
    match (from_format,to_format):
        
        #From MD to HTML, assuming that the content is a single # that will be 
        #replaced with an <h1>
        case (DocFormat.MD, DocFormat.HTML):
            return f"<h1>{content.replace('#','').lstrip()}</h1>"
        #From TXT to PDF. Just adding a pdf flag to the beginning and end of content
        case (DocFormat.TXT, DocFormat.PDF):
            return f"[PDF] {content} [PDF]"
        #From HTML to MD. Doing the inverse than previous
        case (DocFormat.HTML, DocFormat.MD):
            return f"{content.replace('<h1>','# ').replace('</h1>','')}"
            
        #If both formats are the same
        case (first_format,second_format) if first_format == second_format:
            return content
            
        #Any other conversion
        case _:
            raise Exception('invalid type')

In [33]:
try:
    DocFormat.MD and DocFormat.HTML and DocFormat.PDF and DocFormat.TXT
except Exception as error:
    print(f"Error: Missing attribute {error} from enum")

    class DocFormat(Enum):
        PDF = None
        TXT = None
        MD = None
        HTML = None


run_cases = [
    ("# Hello, world!", DocFormat.MD, DocFormat.HTML, "<h1>Hello, world!</h1>"),
    (
        "This is plain text.",
        DocFormat.TXT,
        DocFormat.PDF,
        "[PDF] This is plain text. [PDF]",
    ),
]

submit_cases = run_cases + [
    ("<h1>Title</h1>", DocFormat.HTML, DocFormat.MD, "# Title"),
    ("Something wicked", DocFormat.TXT, None, "invalid type"),
]


def test(content, from_format, to_format, expected_output):
    print("---------------------------------")
    print(f"Converting from {from_format} to {to_format}...")
    print(f"Content: {content}")
    print(f"Expected: {expected_output}")
    try:
        result = convert_format(content, from_format, to_format)
    except Exception as e:
        result = str(e)
    print(f"Actual: {result}")
    if result == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Converting from DocFormat.MD to DocFormat.HTML...
Content: # Hello, world!
Expected: <h1>Hello, world!</h1>
Actual: <h1>Hello, world!</h1>
Pass
---------------------------------
Converting from DocFormat.TXT to DocFormat.PDF...
Content: This is plain text.
Expected: [PDF] This is plain text. [PDF]
Actual: [PDF] This is plain text. [PDF]
Pass
---------------------------------
Converting from DocFormat.HTML to DocFormat.MD...
Content: <h1>Title</h1>
Expected: # Title
Actual: # Title
Pass
---------------------------------
Converting from DocFormat.TXT to None...
Content: Something wicked
Expected: invalid type
Actual: invalid type
Pass
============= PASS ==============
4 passed, 0 failed


# Sum Types Practice

Example using the **match** pattern learned

In [47]:
list_practice = [
            ["Card Name", "Condition", "Value"],
            ["Sparky Mouse", "Fair", 100],
            ["Moist Turtle", "Good", 200],
            ["Burning Lizard", "Very Good", 1000],
            ["Mossy Frog", "Poor", 10],
        ]

def convert_to_CSV(lst):
    return ",".join(lst)
    
#print("\n".join(list(map(convert_to_CSV,list_practice))))

#"Customer ID,Billed,Paid\n1,100,100\n2,400,99\n3,50,25"

def convert_to_str(lst):
    return list(map(str,lst))

#print(list(map(convert_to_str,list_practice)))

def convert_to_str_to_CSV(lst):
    return convert_to_str(lst)

print("\n".join(list(map(convert_to_CSV,list(map(convert_to_str_to_CSV,list_practice))))))

Card Name,Condition,Value
Sparky Mouse,Fair,100
Moist Turtle,Good,200
Burning Lizard,Very Good,1000
Mossy Frog,Poor,10


In [48]:
from enum import Enum

class CSVExportStatus(Enum):
    PENDING = 1
    PROCESSING = 2
    SUCCESS = 3
    FAILURE = 4

#Auxiliar functions for this excersice
#Could do lambda, but it is more legible like this
def convert_lst_to_str(lst):
    return list(map(str,lst)) #Convert from a list of anything to a list of str

def convert_str_to_CSV(lst):
    return ",".join(lst) #Joins list of strings 

def get_csv_status(status, data):

    #only checking the status because depending on it, I will modify the data
    match (status):

        #Returns a tuple with a str and a list of str of the data processed
        case CSVExportStatus.PENDING:
            return "Pending...",list(map(convert_lst_to_str,data))

        #Return a tuple with a str and the data converted from a list of lists of str into one str in CSV
        case CSVExportStatus.PROCESSING:
            return "Processing...","\n".join(list(map(convert_str_to_CSV,data)))
        
        #Return a tuple with a str and the data as is
        case CSVExportStatus.SUCCESS:
            return "Success!",data

        #Return a tuple with a str and the data processed into a CSV str
        case CSVExportStatus.FAILURE:
            #local variables for easy read
            data_as_str = list(map(convert_lst_to_str,data))
            final_csv = "\n".join(list(map(convert_str_to_CSV,data_as_str)))
            
            return "Unknown error, retrying...",final_csv
        
        #Any other case
        case _:
            raise Exception("unknown export status")

In [46]:
try:
    (
        CSVExportStatus.PENDING
        and CSVExportStatus.PROCESSING
        and CSVExportStatus.SUCCESS
        and CSVExportStatus.FAILURE
    )
except Exception as error:
    print(f"Error: Missing attribute {error} from enum")

    class CSVExportStatus(Enum):
        PENDING = None
        PROCESSING = None
        SUCCESS = None
        FAILURE = None


run_cases = [
    (
        CSVExportStatus.PENDING,
        [
            ["Customer ID", "Billed", "Paid"],
            [1, 100, 100],
            [2, 400, 99],
            [3, 50, 25],
        ],
        (
            "Pending...",
            [
                ["Customer ID", "Billed", "Paid"],
                ["1", "100", "100"],
                ["2", "400", "99"],
                ["3", "50", "25"],
            ],
        ),
    ),
    (
        CSVExportStatus.PROCESSING,
        [
            ["Customer ID", "Billed", "Paid"],
            ["1", "100", "100"],
            ["2", "400", "99"],
            ["3", "50", "25"],
        ],
        (
            "Processing...",
            "Customer ID,Billed,Paid\n1,100,100\n2,400,99\n3,50,25",
        ),
    ),
    (
        CSVExportStatus.SUCCESS,
        "Customer ID,Billed,Paid\n1,100,100\n2,400,99\n3,50,25",
        (
            "Success!",
            "Customer ID,Billed,Paid\n1,100,100\n2,400,99\n3,50,25",
        ),
    ),
    (
        CSVExportStatus.FAILURE,
        [
            ["Customer ID", "Billed", "Paid"],
            [1, 100, 100],
            [2, 400, 99],
            [3, 50, 25],
        ],
        (
            "Unknown error, retrying...",
            "Customer ID,Billed,Paid\n1,100,100\n2,400,99\n3,50,25",
        ),
    ),
]

submit_cases = run_cases + [
    (
        CSVExportStatus.PENDING,
        [
            ["Card Name", "Condition", "Value"],
            ["Sparky Mouse", "Fair", 100],
            ["Moist Turtle", "Good", 200],
            ["Burning Lizard", "Very Good", 1000],
            ["Mossy Frog", "Poor", 10],
        ],
        (
            "Pending...",
            [
                ["Card Name", "Condition", "Value"],
                ["Sparky Mouse", "Fair", "100"],
                ["Moist Turtle", "Good", "200"],
                ["Burning Lizard", "Very Good", "1000"],
                ["Mossy Frog", "Poor", "10"],
            ],
        ),
    ),
    (
        CSVExportStatus.PROCESSING,
        [
            ["Card Name", "Condition", "Value"],
            ["Sparky Mouse", "Fair", "100"],
            ["Moist Turtle", "Good", "200"],
            ["Burning Lizard", "Very Good", "1000"],
            ["Mossy Frog", "Poor", "10"],
        ],
        (
            "Processing...",
            "Card Name,Condition,Value\nSparky Mouse,Fair,100\nMoist Turtle,Good,200\nBurning Lizard,Very Good,1000\nMossy Frog,Poor,10",
        ),
    ),
    (
        CSVExportStatus.SUCCESS,
        "Card Name,Condition,Value\nSparky Mouse,Fair,100\nMoist Turtle,Good,200\nBurning Lizard,Very Good,1000\nMossy Frog,Poor,10",
        (
            "Success!",
            "Card Name,Condition,Value\nSparky Mouse,Fair,100\nMoist Turtle,Good,200\nBurning Lizard,Very Good,1000\nMossy Frog,Poor,10",
        ),
    ),
    (
        CSVExportStatus.FAILURE,
        [
            ["Card Name", "Condition", "Value"],
            ["Sparky Mouse", "Fair", 100],
            ["Moist Turtle", "Good", 200],
            ["Burning Lizard", "Very Good", 1000],
            ["Mossy Frog", "Poor", 10],
        ],
        (
            "Unknown error, retrying...",
            "Card Name,Condition,Value\nSparky Mouse,Fair,100\nMoist Turtle,Good,200\nBurning Lizard,Very Good,1000\nMossy Frog,Poor,10",
        ),
    ),
    (1, None, ("Exception Raised:", "unknown export status")),
]


def test(status, data, expected_output):
    print("---------------------------------")
    print(f"Checking: {status}")
    print("Expected:")
    print(f"{expected_output[0]}")
    print(f"{expected_output[1]}")
    try:
        result = get_csv_status(status, data)
    except Exception as e:
        result = expected_output[0], str(e)
    print("Actual:")
    print(f"{result[0]}")
    print(f"{result[1]}")
    if result == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Checking: CSVExportStatus.PENDING
Expected:
Pending...
[['Customer ID', 'Billed', 'Paid'], ['1', '100', '100'], ['2', '400', '99'], ['3', '50', '25']]
Actual:
Pending...
[['Customer ID', 'Billed', 'Paid'], ['1', '100', '100'], ['2', '400', '99'], ['3', '50', '25']]
Pass
---------------------------------
Checking: CSVExportStatus.PROCESSING
Expected:
Processing...
Customer ID,Billed,Paid
1,100,100
2,400,99
3,50,25
Actual:
Processing...
Customer ID,Billed,Paid
1,100,100
2,400,99
3,50,25
Pass
---------------------------------
Checking: CSVExportStatus.SUCCESS
Expected:
Success!
Customer ID,Billed,Paid
1,100,100
2,400,99
3,50,25
Actual:
Success!
Customer ID,Billed,Paid
1,100,100
2,400,99
3,50,25
Pass
---------------------------------
Checking: CSVExportStatus.FAILURE
Expected:
Unknown error, retrying...
Customer ID,Billed,Paid
1,100,100
2,400,99
3,50,25
Actual:
Unknown error, retrying...
Customer ID,Billed,Paid
1,100,100
2,400,99
3,50,25
Pass
-------------